In [1]:
from datasets import load_dataset

dataset = load_dataset("barc0/200k_HEAVY_gpt4o-description-gpt4omini-code_generated_problems")

/mnt/nas/suehyun/ARC/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 199728 examples [01:42, 1957.72 examples/s]


In [3]:
dataset = dataset["train"]

In [5]:
d = dataset[0]

In [6]:
print(d["source"])

from common import *
import numpy as np
from typing import *

# concepts:
# pattern detection, color replacement

# description:
# In the input you will see a grid with several square patterns of two different colors arranged in a checkerboard fashion.
# To make the output, you should replace every second square's color with the color of its adjacent square (to the right) in the checkerboard pattern.

def main(input_grid: np.ndarray) -> np.ndarray:
    output_grid = np.copy(input_grid)

    # Get the dimensions of the input grid
    rows, cols = input_grid.shape

    # Iterate through the grid to find squares in a checkerboard pattern
    for i in range(0, rows, 2):  # Iterate over every second row
        for j in range(0, cols, 2):  # Iterate over every second column
            if i < rows - 1 and j < cols - 1:  # Check bounds
                # Replace the color of the square at (i, j) with the color of the square to the right (i, j+1)
                output_grid[i + 1, j] = input_g

In [7]:
print(d["seeds"][-1])

Concepts: 
repeating patterns, object detection

Description: 
In the input you will see a pattern repeated horizontally and vertically. The pattern includes a square of one color and a rectangle of another color. To make the output, you should detect the pattern and replace the color of the square with the color of the rectangle in each pattern repetition.


In [8]:
len(d["examples"])

30

In [9]:
len(dataset[50]["examples"])

40

In [12]:
from huggingface_hub import snapshot_download
from datasets import load_dataset
import os
import glob

# Download the files
local_dir = snapshot_download(
    repo_id="barc0/induction_samples_with_execution_results",
    repo_type="model"
)

# Get all jsonl files in ARC-Potpourri subdirectory
jsonl_files = glob.glob(os.path.join(local_dir, "ARC-Potpourri", "*.jsonl"))

# Load all files as one dataset
dataset = load_dataset('json', data_files=jsonl_files)

# If you want to check what files were found:
print("Found files:", jsonl_files)

# Print basic info about the dataset
print(dataset)

Fetching 80 files: 100%|██████████| 80/80 [06:39<00:00,  5.00s/it]
Generating train split: 0 examples [00:00, ? examples/s]Failed to load JSON from file '/home/suehyun/.cache/huggingface/hub/models--barc0--induction_samples_with_execution_results/snapshots/c6b9d6414937cf965613c078a739d8a8e191053b/ARC-Potpourri/arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1028185934279481_ae8a984b_exec_results_v5.jsonl' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column(/output_grids/[]/[]) changed from array to string in row 0
Generating train split: 0 examples [00:00, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset

In [43]:
from huggingface_hub import snapshot_download
from datasets import Dataset
import pandas as pd
import os
import glob
import json

# Download the files
local_dir = snapshot_download(
    repo_id="barc0/induction_samples_with_execution_results",
    repo_type="model"
)

# Get all jsonl files
jsonl_files = glob.glob(os.path.join(local_dir, "ARC-Potpourri", "*.jsonl"))

# Function to read jsonl safely
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            try:
                d = json.loads(line.strip())
                # Clean up any mixed types to ensure consistent data structure
                for k, v in d.items():
                    if k in {'uid', 'prompt', 'responses', 'base_model', 'lora_dir'}:
                        continue
                    if k == 'output_grids':
                        # Convert error arrays (arrays of strings) to None
                        # Keep only arrays of arrays (valid grids)
                        if any(isinstance(x, str) for x in v):
                            d[k] = None
                        else:
                            d[k] = v
                    else:
                        # For other arrays (verdicts), convert strings to None
                        for i, v_ in enumerate(v):
                            if isinstance(v_, str):
                                v[i] = None
                    d[k] = v
                data.append(d)
            except json.JSONDecodeError:
                print(f"Skipping malformed JSON in {file_path}")
                continue
    return data

# Combine all data
all_data = []
for file_path in jsonl_files:
    all_data.extend(read_jsonl(file_path))

# Convert to pandas DataFrame
df = pd.DataFrame(all_data)

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

print(dataset)

Fetching 80 files: 100%|██████████| 80/80 [00:00<00:00, 229040.49it/s]


ArrowInvalid: ('cannot mix list and non-list, non-null values', 'Conversion failed for column output_grids with type object')

In [44]:
from huggingface_hub import snapshot_download
import os
import glob
import json
import shutil

# Download the files
local_dir = snapshot_download(
    repo_id="barc0/induction_samples_with_execution_results",
    repo_type="model"
)

# Create output directory
output_dir = "/mnt/nas/suehyun/BARC/induction_samples"
os.makedirs(output_dir, exist_ok=True)

# Get all jsonl files
jsonl_files = glob.glob(os.path.join(local_dir, "ARC-Potpourri", "*.jsonl"))

def clean_and_save_jsonl(input_file_path, output_dir):
    # Get just the filename for output
    filename = os.path.basename(input_file_path)
    output_path = os.path.join(output_dir, filename)
    
    cleaned_data = []
    with open(input_file_path, 'r') as f:
        for line in f:
            try:
                d = json.loads(line.strip())
                cleaned_data.append(d)
            except json.JSONDecodeError:
                print(f"Skipping malformed JSON in {input_file_path}")
                continue
    
    # Write cleaned data to new file
    with open(output_path, 'w') as f:
        for d in cleaned_data:
            f.write(json.dumps(d) + '\n')
    
    print(f"Processed {filename}")

# Process each file
for file_path in jsonl_files:
    clean_and_save_jsonl(file_path, output_dir)

print(f"All files processed and saved to {output_dir}/")

Fetching 80 files: 100%|██████████| 80/80 [00:00<00:00, 21339.63it/s]


Processed arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1028185934279481_ae8a984b_exec_results_v5.jsonl
Processed arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1028191048853424_1c9196fa_exec_results_v5.jsonl
Processed arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1027211442993711_f52a5f0d_exec_results_v5.jsonl
Processed arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1028104645289263_f7dce369_exec_results_v5.jsonl
Processed arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1029015850953489_4bd07868_exec_results_v5.jsonl
Processed arc_problems_validation_400_extra_newline_v2_l3.1-

In [46]:
import os

root = "/mnt/nas/suehyun/BARC/induction_samples"
uids = set()
tolerance = 0
for file in os.listdir(root):
    with open(os.path.join(root, file)) as f:
        for line in f:
            d = json.loads(line)
            uid = d["uid"]
            if uid not in uids:
                uids.add(uid)
            else:
                print(f"{uid} in set")
                tolerance += 1
                if tolerance > 10:
                    break
    if tolerance > 10:
        break

62ab2642 in set
f3cdc58f in set
0692e18c in set
05a7bcf2 in set
2753e76c in set
256b0a75 in set
17cae0c1 in set
c658a4bd in set
79fb03f4 in set
b942fd60 in set
137f0df0 in set


In [26]:
jsonl_files = glob.glob(os.path.join(local_dir, "ARC-Potpourri", "*.jsonl"))
for file in jsonl_files:
    print(os.path.basename(file))

arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1028185934279481_ae8a984b_exec_results_v5.jsonl
arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1028191048853424_1c9196fa_exec_results_v5.jsonl
arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1027211442993711_f52a5f0d_exec_results_v5.jsonl
arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1028104645289263_f7dce369_exec_results_v5.jsonl
arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1029015850953489_4bd07868_exec_results_v5.jsonl
arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_te

In [15]:
from huggingface_hub import hf_hub_download
import json

# Download a single file
file_path = hf_hub_download(
    repo_id="barc0/induction_samples_with_execution_results",
    filename="ARC-Potpourri/arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1027134544075475_8e9068e0_exec_results_v5.jsonl",
    repo_type="model"
)

# Read and examine the data
with open(file_path, 'r') as f:
    # Load first few lines to examine structure
    data = []
    for i, line in enumerate(f):
        if i < 5:  # Look at first 5 records
            record = json.loads(line)
            print(record.keys())
            data.append(record)
            print(f"\nRecord {i} output_grids type:", type(record['output_grids']))
            # print(f"Record {i} output_grids:", record['output_grids'])

# Now we can see what's causing the type mixing issue and handle it appropriately

dict_keys(['uid', 'prompt', 'responses', 'base_model', 'lora_dir', 'train_verdicts', 'train_test_verdicts', 'output_grids', 'verdicts_per_examples'])

Record 0 output_grids type: <class 'list'>
dict_keys(['uid', 'prompt', 'responses', 'base_model', 'lora_dir', 'train_verdicts', 'train_test_verdicts', 'output_grids', 'verdicts_per_examples'])

Record 1 output_grids type: <class 'list'>
dict_keys(['uid', 'prompt', 'responses', 'base_model', 'lora_dir', 'train_verdicts', 'train_test_verdicts', 'output_grids', 'verdicts_per_examples'])

Record 2 output_grids type: <class 'list'>
dict_keys(['uid', 'prompt', 'responses', 'base_model', 'lora_dir', 'train_verdicts', 'train_test_verdicts', 'output_grids', 'verdicts_per_examples'])

Record 3 output_grids type: <class 'list'>
dict_keys(['uid', 'prompt', 'responses', 'base_model', 'lora_dir', 'train_verdicts', 'train_test_verdicts', 'output_grids', 'verdicts_per_examples'])

Record 4 output_grids type: <class 'list'>


In [29]:
with open(file_path) as f:
    data = [json.loads(line) for line in f]

In [42]:
print(len(data))

400


In [30]:
with open("arc_problems_validation_400_extra_newline_v2_l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2_temp_1.0_mpl_16000_tt_1027134544075475_8e9068e0_exec_results_v5.jsonl", 'w') as f:
    for d in data:
        f.write(json.dumps(d) + '\n')

In [25]:
record["base_model"]

'barc0/l3.1-8b-inst-fft-induction-barc-heavy-200k-old-200k-lr1e-5-ep2'

In [27]:
len(record["responses"])

256

In [28]:
print(record["uid"])

2753e76c


In [24]:
print(record["prompt"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a world-class puzzle solver with exceptional pattern recognition skills and expertise in Python programming. Your task is to analyze puzzles and provide Python solutions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Given input-output grid pairs as reference examples, carefully observe the patterns to predict the output grid for new test input. Each pair follows the same transformation rule. Grids are 2D arrays represented as strings, with cells (colors) separated by spaces and rows by newlines.
Here are the input and output grids for the reference examples:
Example 1
Input:
Purple Purple Purple Purple Purple Black Black Black Black Black Black Black Black Black Black Black
Purple Purple Purple Purple Purple Black Black Black Black Black Black Black Black Black Black Black
Purple Purple Purple Purple Purple Black Black Black Purple Purple Purple Purp

In [18]:
print(record["responses"][0])

Let's solve this puzzle using Python code with the common library functions. We'll first reason about the problem and then write the code to solve it. The `transform` function will take the input grid and return the output grid. Here is the Python code with the comments describing how to solve the problem:
```python
from common import *

import numpy as np
from typing import *

# concepts:
# object grouping, color merging

# description:
# In the input you will see several colored circles scattered on a grid. Each circle is of a specific color.
# To create the output grid, the circles need to be grouped by color and merged into larger blocks, 
# with each block assigned to a corner of the output grid based on the color: top-left for red, 
# top-right for green, bottom-left for blue, and bottom-right for yellow.

def transform(input_grid):
    # Detect all colored circles in the input grid
    objects = detect_objects(grid=input_grid, monochromatic=False, connectivity=8)

    # Create a

In [19]:
print(record["train_verdicts"])

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [20]:
print(record["train_test_verdicts"])

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False

In [23]:
print(record["verdicts_per_examples"])

[[None, False, None, False], [False, False, False, False], [False, None, False, None], [False, False, False, False], [False, None, False, False], [False, False, False, False], [False, False, False, False], [False, False, False, False], [False, False, False, None], [False, False, False, False], [None, None, None, None], [False, False, None, False], [None, None, None, None], [False, False, False, False], [False, False, False, False], [False, False, False, False], [False, False, False, False], [None, None, None, None], [False, False, False, False], [False, False, False, False], [False, False, False, False], [False, False, False, False], [False, False, False, False], [False, False, False, False], [None, False, False, False], [False, False, None, False], [None, None, None, None], [False, False, False, False], [False, False, False, False], [False, False, False, False], [None, None, None, None], [False, False, None, False], [None, False, False, False], [None, False, False, False], [False, Fal

In [ ]:
# Look at one file's content
example_file = jsonl_files[0]
data = read_jsonl(example_file)
print("First record structure:")
print(json.dumps(data[0], indent=2))

# Check if all records have the same keys
keys = set(data[0].keys())
for record in data[1:10]:  # Check first 10 records
    keys = keys.union(set(record.keys()))
print("\nAll possible keys:")
print(sorted(list(keys)))

In [11]:
transduction = load_dataset("barc0/transduction_experimental_results")

Generating test split: 100%|██████████| 1676/1676 [00:00<00:00, 6926.23 examples/s]


In [40]:
import numpy as np 

def calculate_normalized_return(verdicts):
    rewards = {None: -2, False: -0.5, True: 1}
    returns = sum(rewards[v] for v in verdicts) 
    return returns / len(verdicts)

def calculate_return_stats(all_verdicts):
    norm_returns = [calculate_normalized_return(verdicts) for verdicts in all_verdicts]
    mean = np.mean(norm_returns)
    std = np.std(norm_returns)
    return mean, std

for d in data:
    mean, std = calculate_return_stats(d["verdicts_per_examples"])
    print(f"{d['uid']}\t{mean:.1f}±{std:.1f}")
    

62ab2642	-0.7±0.5
f3cdc58f	-0.6±0.5
0692e18c	-0.6±0.4
05a7bcf2	-1.6±0.6
2753e76c	-0.7±0.5
256b0a75	-1.8±0.4
17cae0c1	-1.4±0.7
c658a4bd	-0.7±0.5
79fb03f4	-0.7±0.4
b942fd60	-0.6±0.3
137f0df0	-0.7±0.4
7d419a02	-0.9±0.6
281123b4	-1.3±0.7
d282b262	-1.3±0.3
140c817e	-0.6±0.4
414297c0	-1.0±0.6
be03b35f	-1.0±0.7
73c3b0d8	-0.8±0.5
aa300dc3	-0.8±0.6
11e1fe23	-0.9±0.7
c87289bb	-0.9±0.6
6ad5bdfd	-0.7±0.5
9b365c51	-1.1±0.7
8cb8642d	-0.8±0.6
a096bf4d	-0.8±0.6
4cd1b7b2	-0.6±0.4
54db823b	-0.7±0.5
292dd178	-1.1±0.7
8ba14f53	-1.0±0.7
93c31fbe	-1.0±0.6
f3e62deb	-0.6±0.3
e57337a4	-1.3±0.7
e681b708	-0.8±0.6
e5c44e8f	-0.7±0.5
2072aba6	-0.5±0.6
ef26cbf6	-0.8±0.6
c97c0139	-0.8±0.5
642248e4	-1.6±0.7
9772c176	-2.0±0.3
516b51b7	-0.7±0.5
e7639916	-0.7±0.6
bc4146bd	-0.8±0.6
fafd9572	-0.9±0.5
42a15761	-0.9±0.6
c074846d	-1.7±0.3
762cd429	-1.3±0.7
4e45f183	-0.8±0.6
b15fca0b	-0.7±0.5
c6e1b8da	-0.7±0.5
604001fa	-0.7±0.5
81c0276b	-0.8±0.6
103eff5b	-1.9±0.4
ac0c5833	-1.8±0.2
fd096ab6	-1.3±0.7
f45f5ca7	-0.6±0.4
aa4ec2a5	-

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from tqdm import tqdm
from scipy.stats import norm
import random
random.seed(42)
from random import sample

def calculate_normalized_return(verdicts):
    rewards = {None: -2, False: -0.5, True: 1}
    returns = sum(rewards[v] for v in verdicts) 
    return returns / len(verdicts)

def calculate_task_stats(all_verdicts):
    norm_returns = [calculate_normalized_return(verdicts) for verdicts in all_verdicts]
    mean = np.mean(norm_returns)
    std = np.std(norm_returns)
    return mean, std

def categorize_distribution(mean, std, mean_threshold=0.0, std_threshold=0.5):
    if mean > mean_threshold:
        if std > std_threshold:
            return 0  # Positive mean, high std
        else:
            return 1  # Positive mean, low std
    else:
        if std > std_threshold:
            return 2  # Negative mean, high std
        else:
            return 3  # Other (negative mean, low std)

def plot_task_distributions(file_stats, output_path):
    plt.figure(figsize=(15, 8))
    
    # Define colors for each category
    colors = ['#1f77b4',  # Blue: positive mean, high std
             '#2ca02c',   # Green: positive mean, low std
             '#d62728',   # Red: negative mean, high std
             '#7f7f7f']  # Gray: other
    
    x = np.linspace(-2, 1, 300)
    
    # Categorize each distribution
    categorized_stats = []
    for stat in file_stats:
        category = categorize_distribution(stat['mean'], stat['std'])
        categorized_stats.append((stat, category))
    
    # Calculate counts for each category
    counts = [sum(1 for _, cat in categorized_stats if cat == i) for i in range(4)]
    
    # Calculate alpha values inversely proportional to count
    max_count = max(counts)
    alpha_values = [min(0.8, max_count/(count*3)) if count > 0 else 0.8 for count in counts]
    
    category_names = ['Positive μ, High σ',
                     'Positive μ, Low σ',
                     'Negative μ, High σ',
                     'Other']
    
    # Plot individual distributions by category with adjusted alpha
    for cat in range(4):
        first_in_cat = True
        for stat, category in categorized_stats:
            if category == cat:
                label = category_names[cat] if first_in_cat else None
                plt.plot(x, norm.pdf(x, stat['mean'], stat['std']), 
                        alpha=alpha_values[category], color=colors[category], 
                        label=label)
                first_in_cat = False
    
    # Calculate and plot overall distribution
    all_means = [stat['mean'] for stat, _ in categorized_stats]
    overall_mean = np.mean(all_means)
    overall_std = np.std(all_means)
    plt.plot(x, norm.pdf(x, overall_mean, overall_std), 
            color='black', linewidth=2, 
            label=f'Overall (μ={overall_mean:.2f}, σ={overall_std:.2f})')
    
    # Add category counts to legend
    legend_labels = [f'{name} (n={count}, α={alpha_values[i]:.2f})' 
                    for i, (name, count) in enumerate(zip(category_names, counts))]
    
    plt.title(f'Task Return Distributions by Category\n{os.path.basename(output_path)}')
    plt.xlabel('Normalized Return')
    plt.ylabel('Density')
    plt.grid(True, alpha=0.3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

# Create output directory
output_dir = "/mnt/nas/suehyun/BARC/induction_samples/return_distributions"
os.makedirs(output_dir, exist_ok=True)

# Process each file
root = "/mnt/nas/suehyun/BARC/induction_samples"
# samples = sample(os.listdir(root), 10)
samples = os.listdir(root)
all_categorized_stats = []
for file in tqdm(samples):
    file_stats = []
    
    # Process each task
    if not os.path.isfile(file):
        continue
    with open(os.path.join(root, file)) as f:
        for line in f:
            d = json.loads(line)
            mean, std = calculate_task_stats(d["verdicts_per_examples"])
            file_stats.append({
                'uid': d['uid'],
                'mean': mean,
                'std': std
            })
            
    categorized_stats = []
    for stat in file_stats:
        category = categorize_distribution(stat['mean'], stat['std'])
        categorized_stats.append(stat | {"category": category})
    all_categorized_stats.extend(categorized_stats)
    
    # Create plot
#     output_path = os.path.join(output_dir, f"{file.replace('.jsonl', '_categorized.png')}")
#     plot_task_distributions(file_stats, output_path)

# print(f"Plots saved in {output_dir}/")

100%|██████████| 80/80 [00:07<00:00, 10.85it/s]


In [22]:
all_categorized_stats

[{'uid': '62ab2642',
  'mean': -0.70068359375,
  'std': 0.4709034093960618,
  'category': 3},
 {'uid': 'f3cdc58f',
  'mean': -0.60986328125,
  'std': 0.5390764317578106,
  'category': 2},
 {'uid': '0692e18c',
  'mean': -0.61865234375,
  'std': 0.3789134860576776,
  'category': 3},
 {'uid': '05a7bcf2',
  'mean': -1.59130859375,
  'std': 0.6422617402984141,
  'category': 2},
 {'uid': '2753e76c',
  'mean': -0.72705078125,
  'std': 0.5065712651148055,
  'category': 2},
 {'uid': '256b0a75',
  'mean': -1.841796875,
  'std': 0.4362382405179931,
  'category': 3},
 {'uid': '17cae0c1',
  'mean': -1.36015625,
  'std': 0.722175515775727,
  'category': 2},
 {'uid': 'c658a4bd',
  'mean': -0.7265625,
  'std': 0.49945038151326904,
  'category': 3},
 {'uid': '79fb03f4',
  'mean': -0.6958705357142856,
  'std': 0.44863260339707306,
  'category': 3},
 {'uid': 'b942fd60',
  'mean': -0.5962611607142857,
  'std': 0.34701196372538895,
  'category': 3},
 {'uid': '137f0df0',
  'mean': -0.6787109375,
  'std': 0.

In [23]:
import pandas as pd 

df = pd.DataFrame(all_categorized_stats)
df.describe()

,mean,std,category
count,400.000000,400.000000,400.000000
mean,-0.902633,0.544222,2.357500
std,0.408327,0.143554,0.538906
min,-1.975586,0.123465,0.000000
25%,-1.030713,0.448569,2.000000
50%,-0.791650,0.540350,2.000000
75%,-0.662598,0.638724,3.000000
max,0.743652,1.158770,3.000000


In [24]:
df.groupby(by="category").describe()

mean                                                              \
          count      mean       std       min       25%       50%       75%   
category                                                                      
0           4.0  0.585083  0.162117  0.408203  0.468628  0.594238  0.710693   
2         245.0 -0.970623  0.330860 -1.794922 -1.116699 -0.873047 -0.780078   
3         151.0 -0.831729  0.439633 -1.975586 -0.718848 -0.668457 -0.616943   

                      std                                                    \
               max  count      mean       std       min       25%       50%   
category                                                                      
0         0.743652    4.0  0.698635  0.099157  0.571880  0.644129  0.718369   
2        -0.022461  245.0  0.628838  0.098546  0.501839  0.552806  0.616864   
3        -0.204102  151.0  0.402842  0.079818  0.123465  0.369805  0.424994   

                              
               75%       max  
category                      
0         0.772875  0.785921  
2         0.680203  1.158770  
3         0.463870  0.499784